In [1]:
from U2GNN_pytorch import util

In [22]:
data = util.load_data("PTC",False, ".")

loading data
# classes: 2
# feature_size: 19
# max nodes : 109
# min nodes : 2
# maximum node tag: 19
# data: 344


In [23]:
data[1]

2

In [11]:
data[0][0].__dict__

{'label': 0,
 'g': <networkx.classes.graph.Graph at 0x7f23bc2a8780>,
 'node_tags': [0, 1, 0, 0, 2],
 'neighbors': [[1], [0, 2, 3, 4], [1], [1], [1]],
 'node_features': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]], dtype=float32),
 'edge_mat': array([[0, 1, 1, 1, 1, 2, 3, 4],
        [1, 2, 3, 4, 0, 1, 1, 1]], dtype=int32),
 'max_neighbor': 4,
 'knn_g': None,
 'attr_gs': [],
 'edge_mat_knn': None,
 'edge_mat_attrs': []}

In [12]:
import dgl

Using backend: pytorch


In [13]:
data = dgl.data.TUDataset('DD')

AttributeError: module 'dgl' has no attribute 'data'

In [14]:
dgl.__version__

'0.4.3post2'

In [15]:
from dgl.data import TUDataset

In [24]:
data_dgl = TUDataset('AIDS')

In [25]:
data_dgl.__dict__.keys()

dict_keys(['name', 'extract_dir', 'max_num_node', 'num_labels', 'graph_labels', 'attr_dict', 'graph_lists'])

In [61]:
#data_dgl.__dict__["graph_lists"][0].ndata["node_labels"]
data_dgl.__dict__["attr_dict"]

{'node_labels': ('ndata', 'node_labels'),
 'node_attributes': ('ndata', 'node_attr'),
 'edge_labels': ('edata', 'edge_labels'),
 'edge_attributes': ('edata', 'node_labels')}

In [98]:
data_dgl.num_labels[0].item()

2

In [47]:
"node_attr" in data_dgl.__dict__["graph_lists"][0].ndata.keys()

False

In [110]:
import numpy as np
from U2GNN_pytorch import util
def get_onehot_features(graph_info,max_node_tag):
    feature_size = max_node_tag
    '''
    if("node_attr" in graph_info.ndata.keys()):
        feature_size += graph_info.ndata["node_attr"].shape[1]
    np_a = np.zeros(feature_size )
    '''
    node_feats = []
    for node in range(graph_info.ndata["node_labels"].shape[0]):
        cur_feat = np.zeros(feature_size)
        cur_label = graph_info.ndata["node_labels"][node].item()
        cur_feat[cur_label] = 1
        assert cur_feat.nonzero()[0] == cur_label
        node_feats.append(cur_feat)
    
    return np.stack(node_feats,axis = 0)


def make_edge_mat_neighs(g_svg):
        #add labels and edge_mat       
    g = g_svg
    g.neighbors = [[] for i in range(len(g.g))]
    for i, j in g.g.edges():
        g.neighbors[i].append(j)
        g.neighbors[j].append(i)
    degree_list = []
    for i in range(len(g.g)):
        g.neighbors[i] = g.neighbors[i]
        degree_list.append(len(g.neighbors[i]))
    g.max_neighbor = max(degree_list)
    
    

    edges = [list(pair) for pair in g.g.edges()]
    edges.extend([[i, j] for j, i in edges])

    deg_list = list(dict(g.g.degree(range(len(g.g)))).values())

    g.edge_mat = np.transpose(np.array(edges, dtype=np.int32), (1,0))

    

In [111]:

def load_tu_dataset(dataset_name):
    data_dgl = TUDataset(dataset_name)
    node_labels = False
    if("node_labels" in data_dgl.__dict__["attr_dict"].keys()):
        node_labels = True
    if(node_labels):
        num_labels = data_dgl.num_labels[0]
        max_node_tag = 0
        for g_info in data_dgl.__dict__['graph_lists']:
            cur_tag= g_info.ndata['node_labels'].max().item()+1
            if(cur_tag > max_node_tag ):
                max_node_tag = cur_tag
                #print(cur_tag)

    
    g_list = []
    max_nodes = 0
    min_nodes = 1000000
    for g_l , g_info in zip(data_dgl.graph_labels, data_dgl.graph_lists):
        node_feats = None
        node_tags = None
        g_nx = g_info.to_networkx()
        
        if(node_labels):
            #print(g_info.__dict__)
            node_feats = get_onehot_features(g_info, max_node_tag)
            node_tags = g_info.ndata["node_labels"].numpy().flatten()
            
        if("node_attr" in g_info.ndata.keys()):
            node_feats = np.concatenate([node_feats,g_info.ndata["node_attr"].numpy()],axis = 1)
        l = g_l[0]
        g_svg = util.S2VGraph(g_nx, l, node_tags)
        g_svg.node_features = node_feats
        #g_svg.neighbors = l_neighs
        make_edge_mat_neighs(g_svg)
        #print(g_svg.__dict__)
        
        g_list.append(g_svg)
        if(g_svg.node_features.shape[0]>max_nodes):
            max_nodes = g_svg.node_features.shape[0]
        if(g_svg.node_features.shape[0]<min_nodes):
            min_nodes = g_svg.node_features.shape[0]
    
    print('# classes: %d' % data_dgl.num_labels[0].item())
    print('# feature_size: %d' % g_list[0].node_features.shape[1])
    print('# max nodes : %d' % max_nodes)
    print('# min nodes : %d' % min_nodes)
    
    print("# data: %d" % len(g_list))
    return g_list, data_dgl.num_labels[0].item()


In [102]:
res = load_tu_dataset("AIDS")

2000

In [104]:
res[0][0].__dict__

{'label': 0,
 'g': <networkx.classes.multidigraph.MultiDiGraph at 0x7f23248ab860>,
 'node_tags': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0]),
 'neighbors': [[1, 5, 1, 5],
  [0, 0, 2, 11, 2, 11],
  [1, 1, 3, 14, 3, 14],
  [2, 2, 4, 6, 4, 6],
  [3, 3, 9, 5, 5, 9],
  [0, 4, 0, 4],
  [3, 3, 7, 7],
  [6, 6, 8, 10, 8, 10],
  [7, 7, 45, 9, 9, 45],
  [4, 8, 4, 8],
  [7, 7],
  [1, 1, 12, 12],
  [11, 11, 13, 26, 27, 13, 26, 27],
  [12, 12, 37, 14, 14, 37],
  [2, 13, 2, 13, 24, 24],
  [16, 20, 21, 23, 16, 20, 21, 23],
  [15, 15, 17, 17],
  [16, 16, 18, 18],
  [17, 17, 19, 43, 21, 19, 21, 43],
  [18, 18, 22, 20, 20, 22],
  [15, 19, 15, 19],
  [15, 18, 15, 18, 41, 42, 41, 42],
  [19, 19],
  [15, 15, 25, 24, 24, 25],
  [14, 23, 14, 23],
  [23, 23],
  [12, 12],
  [12, 12],
  [29, 33, 34, 36, 29, 33, 34, 36],
  [28, 28, 30, 30],
  [29, 29, 31, 31],
  [30, 30, 32, 44, 34, 32, 34, 44]